<a href="https://colab.research.google.com/github/WanQi-K/DataSciencePortfolio/blob/main/Part_4_Full_Code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Comparison against the Actual Listing Price on Property Guru

In [ ]:
import numpy as np
import pandas as pd
import scipy
import statsmodels.api as sm
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
sns.set()

#Load Data
filepath=r'C:\Users\Wan Qi\Desktop\2024\05 l Data Science Projects\MY Property\NAPIC Selected Data.csv'
df=pd.read_csv(filepath)

#Clean Data
#Clean up column names (removing any leading/trailing spaces)
df.columns=df.columns.str.strip()

#Transform into Date
df['Transaction Date']=pd.to_datetime(df['Month, Year of Transaction Date'],format='%b-%y')

#Remove RM and , in Transaction Price
df['Transaction Price']=df['Transaction Price'].str.replace('RM','',regex=False)

# Remove "RM", commas, and whitespace from the Transaction Price column
df['Transaction Price'] = df['Transaction Price'].str.replace('RM', '', regex=False)
df['Transaction Price']=df['Transaction Price'].str.replace(',','',regex=False)
df['Transaction Price']=df['Transaction Price'].str.strip().astype(float)

#Replace - with 0
df['Main Floor Area']=df['Main Floor Area'].str.replace('-','',regex=False)
df['Unit']=df['Unit'].str.replace('-','',regex=False)

#Drop unncessary columns
df.drop(columns='Month, Year of Transaction Date',inplace=True)
df.drop(columns='Land/Parcel Area',inplace=True)
df.drop(columns='Unit Metrics',inplace=True)
df.drop(columns='Main Floor Area',inplace=True)
df.drop(columns='Unit',inplace=True)

#Change Unit Level into numeric
df['Unit Level'].str.strip()
df['Unit Level']=df['Unit Level'].str.replace('A','',regex=False)
df['Unit Level']=df['Unit Level'].str.replace('UG','1',regex=False)
df['Unit Level']=df['Unit Level'].str.replace('P','100',regex=False)

# Replace both empty strings and NaN values with 0
df['Unit Level'] = df['Unit Level'].replace('', 0).fillna(0)

# Convert 'Unit Level' to numeric after replacements
df['Unit Level'] = pd.to_numeric(df['Unit Level'], errors='coerce')

#Identify PSF
df['PSF']=df['Transaction Price']/df['Square Feet']

#Extract Month & Date
df['Year']=pd.to_datetime(df['Transaction Date']).dt.year
df['Month']=pd.to_datetime(df['Transaction Date']).dt.month

#Analysing Indera Subang
InderaSubang_data=df[df['Scheme Name/Area']=="INDERA SUBANG KONDOMINIUM - USJ 6"]

df.sort_values(by='Transaction Date',ascending=True, inplace=True)

In [ ]:
#Price Prediction
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from scipy.stats import t

# Add 'Level' feature to the data
# Split data with and without 'Level' feature
X_with_level = InderaSubang_data[["Square Feet", "Unit Level"]]
y = InderaSubang_data["Transaction Price"]

# Train-Test Split for both scenarios
X_train_with, X_test_with, y_train_with, y_test_with = train_test_split(X_with_level, y, test_size=0.2, random_state=42)

# Train models
model_with_level = LinearRegression()
model_with_level.fit(X_train_with, y_train_with)

def predict_price_ci_with_level(square_feet, level, model, SE, t_critical):
    example_input = pd.DataFrame({'Square Feet': [square_feet], 'Unit Level': [level]})
    predicted_price = model.predict(example_input)[0]

    # Confidence Interval
    margin_of_error = t_critical * SE
    lower_bound = predicted_price - margin_of_error
    upper_bound = predicted_price + margin_of_error

    return predicted_price, lower_bound, upper_bound

# Add 'Level' to confidence interval calculations
confidence_level = 0.90  # 90% CI
residuals_with_level = y_train_with - model_with_level.predict(X_train_with)
std_residuals_with_level = np.std(residuals_with_level)
N_with_level = len(y_train_with)
SE_with_level = std_residuals_with_level / np.sqrt(N_with_level)
t_critical_with_level = t.ppf((1 + confidence_level) / 2, N_with_level - 2)

# Example prediction with 'Level'
square_feet = 1711 #Input Square Feet
level = 8  # Input level
predicted_price_with_level, lower_bound_with_level, upper_bound_with_level = predict_price_ci_with_level(
    square_feet, level, model_with_level, SE_with_level, t_critical_with_level
)

# Display results
print(f"Predicted Price for {square_feet} square feet on Level {level}: RM {predicted_price_with_level:,.0f}")
print(f"90% Confidence Interval: RM {lower_bound_with_level:,.0f} - RM {upper_bound_with_level:,.0f}")


Predicted Price for 1711 square feet on Level 8: RM 695,633
90% Confidence Interval: RM 672,433 - RM 718,833


In [ ]:
#Import Another Source (Webscrapping from Property Guru)
Actuallisting_filepath=r'C:\Users\Wan Qi\Desktop\2024\05 l Data Science Projects\MY Property\Property Guru - Indera Subang Listing Data.csv'
Actual_Listing_df=pd.read_csv(Actuallisting_filepath)

#Transform into Date
Actual_Listing_df.columns=Actual_Listing_df.columns.str.strip()

In [ ]:
#Identify if the actual listing price is overpriced, underpriced or within range
# Assuming `model`, `SE`, and `t_critical` are already calculated as in the provided snippet.

# Function to classify price category
def categorize_price(listed_price, lower_bound, upper_bound):
    if lower_bound <= listed_price <= upper_bound:
        return "Within Range"
    elif listed_price > upper_bound:
        return "Overpriced"
    else:
        return "Underpriced"

# Apply predictions and confidence intervals to each row in `Actual_Listing_df`
def add_price_predictions(df, model_with_level, SE_with_level, t_critical_with_level):
    # Initialize new columns
    df['Lower Price Range'] = np.nan
    df['Upper Price Range'] = np.nan
    df['Price Category'] = ""

    for index, row in df.iterrows():
        square_feet = row['Square Feet']
        level = row.get('Unit Level', 0)  # Default to 0 if 'Unit Level' is missing
        listed_price = row['Listed Price']

        # Predict price and confidence interval
        predicted_price, lower_bound, upper_bound = predict_price_ci_with_level(
            square_feet, level, model_with_level, SE_with_level, t_critical_with_level
        )

        # Update DataFrame
        df.at[index, 'Lower Price Range'] = lower_bound
        df.at[index, 'Upper Price Range'] = upper_bound
        df.at[index, 'Price Category'] = categorize_price(listed_price, lower_bound, upper_bound)

    return df

# Add in the additional columns
Actual_Listing_df = add_price_predictions(Actual_Listing_df, model_with_level, SE_with_level, t_critical_with_level)

# Display the updated DataFrame
Actual_Listing_df.sort_values(by='Price Category',ascending=False, inplace=True)
print(Actual_Listing_df)

    Listed On  Listed Price  Square Feet  Bedroom  Bathroom  \
13  29-Oct-24        740000         2185        4         4   
0   29-Nov-24        720000         1485        4         3   
1   26-Nov-24        690000         1537        3         3   
2   26-Nov-24        900000         2500        5         5   
3   24-Nov-24        720000         1711        3         4   
4   16-Nov-24        780000         1776        4         5   
5   15-Nov-24        900000         2500        5         5   
6    6-Nov-24        750000         1711        3         4   
7    6-Nov-24        950000         2500        4         4   
8    6-Nov-24        760000         1485        4         4   
9    5-Nov-24        690000         1537        4         3   
10   5-Nov-24        773000         1776        4         3   
11   5-Nov-24        755000         1485        4         3   
12   4-Nov-24        890000         2500        5         5   
14  17-Oct-24        720000         1780        4      

In [ ]:
#Count by Price Category in Table
# Group by the 'Price Category' column and count the number of transaction prices
listing_by_category = Actual_Listing_df.groupby('Price Category')['Listed On'].count()

# Convert the result to a DataFrame
listing_by_category_count_df=listing_by_category.reset_index()

# Rename the columns for better readability
listing_by_category_count_df.columns=['Price Category','Listing Count']

# Calculate the grand total of all listings
grand_total = listing_by_category_count_df['Listing Count'].sum()
listing_by_category_count_df['Percentage'] = (listing_by_category_count_df['Listing Count'] / grand_total) * 100
listing_by_category_count_df['Percentage'] = listing_by_category_count_df['Percentage'].apply(lambda x: f"{x:.2f}%")

# Display the updated DataFrame
listing_by_category_count_df

,Price Category,Listing Count,Percentage
0,Overpriced,15,93.75%
1,Within Range,1,6.25%
